<a href="https://colab.research.google.com/github/WellcomePeujio/Optimizacion-de-Portafolio-con-Inteligencia-Artificial/blob/main/Optimizaci%C3%B3n_de_Portafolio_con_Inteligencia_Artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización de Portafolio con Inteligencia Artificial

## **Título**: "¿Puede la IA crear el portafolio perfecto?"

### **Problema**  
Se dispone de un conjunto de **activos financieros**, cada uno con su propio nivel de **rendimiento esperado** y **riesgo (volatilidad)**. El objetivo es encontrar la **combinación óptima de pesos** que maximice el rendimiento ajustado al riesgo.

Para ello, se propone utilizar técnicas de **Inteligencia Artificial**, como **algoritmos genéticos** o **redes neuronales**, con el fin de buscar una distribución eficiente del portafolio.

---

### **Preguntas a Resolver**  
1️⃣ Utiliza un **algoritmo genético** o una **red neuronal** para **optimizar la distribución del portafolio**.  
2️⃣ Evalúa el **Sharpe Ratio** y la **volatilidad total** del portafolio óptimo.  
3️⃣ Genera una **visualización de la frontera eficiente**, mostrando diferentes combinaciones de portafolio y destacando el óptimo.

---

### **Fórmulas Clave**

- **Rendimiento del portafolio**:

$$
E(R_p) = \sum_{i=1}^{N} w_i \cdot E(R_i)
$$

- **Volatilidad del portafolio**:

$$
\sigma_p = \sqrt{w^T \cdot \Sigma \cdot w}
$$

- **Sharpe Ratio**:

$$
S = \frac{E(R_p) - R_f}{\sigma_p}
$$

Donde:
- $w_i$ = peso del activo $i$
- $E(R_i)$ = rendimiento esperado del activo $i$
- $\Sigma$ = matriz de covarianza de los activos
- $R_f$ = tasa libre de riesgo

---


# Resolución Matemática: Optimización de Portafolio con Inteligencia Artificial

## Paso 1: Definición del Problema

Queremos encontrar la mejor combinación de pesos $w_i$ para un conjunto de $N$ activos que maximice el rendimiento ajustado al riesgo.

El objetivo es **maximizar el Sharpe Ratio**:

$$
\text{Maximizar} \quad S = \frac{E(R_p) - R_f}{\sigma_p}
$$

Sujeto a:

$$
\sum_{i=1}^{N} w_i = 1 \quad \text{y} \quad w_i \geq 0 \quad \forall i
$$

---

## Paso 2: Cálculo del Rendimiento Esperado del Portafolio

El **rendimiento esperado** del portafolio se calcula como:

$$
E(R_p) = \sum_{i=1}^{N} w_i \cdot E(R_i)
$$

Donde:
- $E(R_i)$ es el rendimiento esperado del activo $i$
- $w_i$ es la proporción del capital invertida en el activo $i$

---

## Paso 3: Cálculo de la Volatilidad del Portafolio

La **volatilidad total** del portafolio se obtiene mediante:

$$
\sigma_p = \sqrt{w^T \cdot \Sigma \cdot w}
$$

Donde:
- $\Sigma$ es la **matriz de covarianza** entre los activos
- $w$ es el **vector de pesos** del portafolio

---

## Paso 4: Cálculo del Sharpe Ratio

El **Sharpe Ratio** es una medida del rendimiento ajustado al riesgo:

$$
S = \frac{E(R_p) - R_f}{\sigma_p}
$$

Donde:
- $R_f$ es la **tasa libre de riesgo**

El objetivo será encontrar el vector de pesos $w$ que maximiza $S$ usando métodos de **Inteligencia Artificial**, como **algoritmos genéticos** o **redes neuronales**.

---

## Paso 5: Implementación de la Frontera Eficiente

Una vez optimizado el portafolio, se puede construir la **Frontera Eficiente**, que representa gráficamente las mejores combinaciones de activos que ofrecen el **máximo rendimiento para un nivel de riesgo dado**.

Este gráfico permite comparar diferentes combinaciones y destacar el **portafolio óptimo** identificado por la IA.

---


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Selección de activos
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
start_date = '2022-01-01'
end_date = '2023-12-31'

# Descarga de precios ajustados
data = yf.download(tickers, start="2019-01-01", end="2024-01-01")['Close']

returns = data.pct_change().dropna()

# Estadísticas del portafolio
mean_returns = returns.mean() * 252  # Rendimiento anualizado
cov_matrix = returns.cov() * 252  # Covarianza anualizada
num_assets = len(tickers)
rf = 0.03  # Tasa libre de riesgo anual

# Función para calcular estadísticas del portafolio
def portfolio_stats(weights):
    port_return = np.dot(weights, mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (port_return - rf) / port_volatility
    return port_return, port_volatility, sharpe_ratio

# Función objetivo para minimizar (negativo del Sharpe Ratio)
def negative_sharpe(weights):
    return -portfolio_stats(weights)[2]

# Restricciones y límites
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(num_assets))
initial_weights = np.ones(num_assets) / num_assets

# Optimización
opt_result = minimize(negative_sharpe, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
opt_weights = opt_result.x
opt_return, opt_volatility, opt_sharpe = portfolio_stats(opt_weights)

# Simulación de portafolios para graficar la frontera eficiente
num_portfolios = 5000
all_weights = np.random.dirichlet(np.ones(num_assets), num_portfolios)
all_returns = []
all_vols = []
all_sharpes = []

for w in all_weights:
    r, v, s = portfolio_stats(w)
    all_returns.append(r)
    all_vols.append(v)
    all_sharpes.append(s)

# Gráfico de la Frontera Eficiente
plt.figure(figsize=(10, 6))
scatter = plt.scatter(all_vols, all_returns, c=all_sharpes, cmap='viridis', alpha=0.5)
plt.colorbar(scatter, label='Sharpe Ratio')
plt.scatter(opt_volatility, opt_return, c='red', s=100, label='Portafolio Óptimo', edgecolors='black')
plt.xlabel('Volatilidad (Riesgo)')
plt.ylabel('Rendimiento Esperado')
plt.title('Frontera Eficiente con Optimización por Sharpe Ratio')
plt.legend()
plt.grid(True)
plt.show()

# Mostrar resultados del portafolio óptimo
opt_weights_dict = dict(zip(tickers, np.round(opt_weights, 4)))
(opt_return, opt_volatility, opt_sharpe), opt_weights_dict